### README
- From the user favourites page in [つくおき](https://cookien.com/favorite/)
- Extract all recipes found inside
  - Title
  - Storage period
  - Ingredients
  - Instructions
---

In [1]:
OUTPUT_LEVEL = 2 # 1 - Ingredients only / 2 - All

### Init selenium stuff

In [2]:
import os.path

from dotenv import load_dotenv
load_dotenv()

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import chromedriver_binary

In [3]:
# Init chromedriver
chrome_options = Options()
chrome_options.add_argument("--headless") # turn off GUI
chrome_options.add_argument("--no-sandbox")

homedir = os.path.expanduser("~")
# webdriver_service = Service(f"{homedir}/chromedriver/stable/chromedriver")

# Init browser
# browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)
browser = webdriver.Chrome(options=chrome_options)



---

### Open favorites page

In [4]:
browser.get("https://cookien.com/favorite/")

elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='wpmem_login']")))
print(browser.title)

print(elems[0].find_element(By.XPATH, "//div[@id='wpmem_login']").text)

| つくおき | 作り置き・簡単レシピサイト
ログイン
ユーザID または メールアドレス
パスワード
 ログイン状態を保存する  
パスワードをお忘れですか？ パスワードリセット
はじめての方はこちら 新規ユーザー登録


---
### Handle login

In [5]:
username = os.getenv("COOKIEN_USER")
password = os.getenv("COOKIEN_PASS")

In [6]:
# Login
login_username = elems[0].find_element(By.XPATH, "//input[@id='log']")
login_password = elems[0].find_element(By.XPATH, "//input[@id='pwd']")
login_submit = elems[0].find_element(By.XPATH, "//input[@type='submit'][@class='buttons']")

login_username.send_keys(username)
login_password.send_keys(password)

login_submit.click()

# Post-login / wait for page to load
print(browser.title)
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='site-info clear']")))
print(elems[0].find_element(By.XPATH, "//div[@class='site-info clear']").text)

お気に入り一覧 | つくおき | 作り置き・簡単レシピサイト
© 2013-2023 つくおき All rights reserved.
当サイトの情報及び、使用している画像、テキスト等の無断転載を禁じます。


--- 

### Extract recipe data for each item found in favourites page

In [7]:
elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='articleList']")))
ingredients = []
for elem in elems[0].find_elements(By.XPATH, ".//a[@class='content_list_link']"):
    elem_text = elem.get_attribute("title")
    elem_url = elem.get_attribute("href")
    if OUTPUT_LEVEL >= 1: 
        print(f"# [{elem_text}]({elem_url})")

    # Open url in new tab
    original_window = browser.current_window_handle
    assert len(browser.window_handles) == 1
    browser.switch_to.new_window('tabaa')
    browser.get(elem_url)  

    # Extract elements
    sub_elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='content']")))

    # Storage period
    if OUTPUT_LEVEL > 1:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='clock_reizouko']").text)
    
    # Ingredients
    if OUTPUT_LEVEL > 0:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='r_contents']/h2").text.replace('\n', ', '))
        for ingredient in sub_elems[0].find_elements(By.XPATH, ".//div[@id='r_contents']/p"):
            print("\t", ingredient.text.replace('\n', ' '))
            ingredients.append(ingredient.text.replace('\n', ' '))

    # Instructions
    if OUTPUT_LEVEL > 1:
        print("##", sub_elems[0].find_element(By.XPATH, ".//div[@id='ins_contents']/h2").text.replace('\n', ', '))
        for instruction in sub_elems[0].find_elements(By.XPATH, ".//p[@class='ins_des']"):
            print("\t", instruction.text.replace('\n', ' '))

    # Close tab and go back to main window
    browser.close()
    browser.switch_to.window(original_window)
    print(50*"=")

# TODO: Automatically upload to Trello / Google sheets?

# [基本の肉じゃが](https://cookien.com/recipe/4184/)
## 30分日持ち：冷蔵5日
## 材料（保存容器中１個分）, 食べきりの場合 ３～４人分
	 豚こま切れ肉（メモ１） 約１５０ｇ
	 じゃがいも １～２個（約２５０ｇ）
	 にんじん 小１本（約１００ｇ）
	 たまねぎ １／２個（約１５０ｇ）
	 ゆで絹さや １０枚（お好みで）
	 ◎砂糖 大３
	 ◎醤油 大２
	 ◎水 １００ｍＬ
## 作り方
	 じゃがいもは皮をむいて芽を取り、食べやすい大きさに切り、２～３分ほど水にさらします。にんじんは薄く半月切り、またはいちょう切りにします。たまねぎは薄切りにします（メモ２）。
	 豚肉はフォークなどで数カ所穴をあけ、食べやすい大きさに切ります。
	 鍋に水気を切ったじゃがいも、にんじん、たまねぎ、豚肉を順番に入れます。◎を入れ、ふたをして煮立たせます。煮立ったら弱火で１５分ほど煮ます（メモ３）。
	 ふたを取り、絹さやを入れ、煮汁がほどよく少なくなるまで煮込み、完成（メモ４）。
# [フライパンでかんたん揚げ出し豆腐](https://cookien.com/recipe/41961/)
## 15分日持ち：冷蔵3日
## 材料（保存容器大1個分）, 食べきりの場合 3～4人分
	 絹ごし豆腐 1丁（350g）
	 ◯片栗粉 大2〜3
	 ◎みりん 大2
	 ◎白だし 大1
	 ◎醤油 大1/2
	 ◎水 150mL
## 作り方
	 豆腐は6等分に切り、水切りします。（メモ1）
	 電子レンジ対応の大きめの保存容器に◎を入れ、500wの電子レンジで2分加熱します。（メモ2）
	 豆腐に片栗粉をまんべんなくまぶします。フライパンに多めの油（大4～5）を弱めの中火で熱します。豆腐をやさしく入れ、揚げ焼きにします。熱いうちに2の保存容器に入れて、完成。（メモ3）
	 食べるときに、お好みで大根おろしや小ねぎをそえても。
# [ほうれん草とたまねぎのオープンオムレツ](https://cookien.com/recipe/2468/)
## 30分日持ち：冷蔵4日
## 材料（保存容器大1個分）, 食べきりの場合 3～4人分
	 卵 4個
	 ほうれん草 1袋（6～8株）
	 たまねぎ 1/2個
	 塩（塩ゆで用） 小1
	 水 1

In [8]:
browser.quit()

--- 

### Extract ingredients data and group same items

In [11]:
import re

parsed_items = {}
skipped_items = {}

for item in ingredients:
    item = re.sub(r'\（メモ\d+\）', '', item)
    item = item.replace('◎', '')
    item = item.replace('◯', '').strip()

    key, value = item.split(' ', 1) 
    key = key.strip()
    value = value.strip()
    
    if not any(ext in item for ext in ["油", "酢", "塩", "みりん", "みそ", "汁", "ソース", "ペッパー", "素", "コンソメ", "パウダー", "砂糖", "だし", "マヨネーズ", "ケチャップ", "酒"]):
        if key in parsed_items:
            parsed_items[key] += f" {value}"
        else:
            parsed_items[key] = value    
    else:
        if key in skipped_items:
            skipped_items[key] += f" {value}"
        else:
            skipped_items[key] = value

result_list = [f"{key} {value}" for key, value in parsed_items.items()]

# Paste entire chunk into a checklist item in Trello (trello will automatically create an item for each line)
for r in result_list:
    print(r)

豚こま切れ肉 約１５０ｇ
じゃがいも １～２個（約２５０ｇ）
にんじん 小１本（約１００ｇ） 1/2本（約60g） 1/3本 小１本（約１００ｇ） １／３本（約５０ｇ）
たまねぎ １／２個（約１５０ｇ） 1/2個 １個（約３００ｇ） １／２個（約１５０ｇ）
ゆで絹さや １０枚（お好みで）
水 １００ｍＬ 150mL 1L １Ｌ 150mL 大2
絹ごし豆腐 1丁（350g）
片栗粉 大2〜3 大1
卵 4個 １個 4個
ほうれん草 1袋（6～8株） 1袋（6～8株）
ミックスチーズ 適量
すりごま 大2
ちくわ 3本（約90g）
しらたき 1袋（約200g）
ピーマン 3個
もやし １袋（２００ｇ）
乾燥わかめ 約４ｇ
いりごま 小１
にんにくチューブ ３ｃｍ
豚ひき肉 約３５０ｇ 約350g 約３００ｇ
にんにく １かけ
トマト缶 １缶
赤ワイン ５０ｍＬ
長ねぎ（白い部分） 2本
しょうが 1かけ（約10g）
木綿豆腐 2丁
セロリ（茎） １本
パン粉 １／２カップ
スパイス お好みで
鶏もも肉 約250g
玉ねぎ 1/2個
大豆水煮 1袋（固形量200g）
ちりめんじゃこ 約30g


In [10]:
skipped_items

{'砂糖': '大３ 大1/2 大2 大1',
 '醤油': '大２ 大1/2 大1/2 大1/2 小１ 小1 大1.5 小2',
 'みりん': '大2 大2 大2',
 '白だし': '大1 大1',
 '塩（塩ゆで用）': '小1 小1',
 '顆粒コンソメ': '小1',
 '塩': '少々 少々 小１／２ 小１／２',
 'オイスターソース': '大1/2 大2',
 'ブラックペッパー': '少々 少々 少々',
 '穀物酢': '大１／２',
 'ごま油': '大１',
 '中華スープの素': '小１．５ 7.5g',
 '中濃ソース': '１００ｍＬ 大１',
 '豆みそ': '大2'}